<a href="https://colab.research.google.com/github/hwasol/esg/blob/main/%ED%99%98%EA%B2%BD%EA%B0%80%EC%B9%98_%EC%A7%80%EC%9D%B4%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()


Saving ND-GAIN 적응 지수 2011.csv to ND-GAIN 적응 지수 2011 (1).csv
Saving 아프리카 재생에너지 비율 2011.csv to 아프리카 재생에너지 비율 2011 (1).csv
Saving 아프리카_CO2_배출량_2010^F2022 (1).csv to 아프리카_CO2_배출량_2010^F2022 (1) (1).csv
Saving 아프리카_CO2_배출량_변화율_2011.csv to 아프리카_CO2_배출량_변화율_2011 (1).csv
Saving 아프리카24개국_보호구역비율_2011.csv to 아프리카24개국_보호구역비율_2011 (1).csv
Saving 아프리카24개국_산림면적_2011.csv to 아프리카24개국_산림면적_2011 (1).csv
Saving 아프리카24개국_환경규제강도_2011.csv to 아프리카24개국_환경규제강도_2011 (1).csv


In [6]:
import os
print(os.listdir('/content'))


['.config', 'ND-GAIN 적응 지수 2011 (1).csv', '아프리카24개국_산림면적_2011 (1).csv', '아프리카24개국_산림면적_2011.csv', '아프리카24개국_환경규제강도_2011 (1).csv', '아프리카 재생에너지 비율 2011 (1).csv', '아프리카_CO2_배출량_변화율_2011.csv', '아프리카24개국_보호구역비율_2011.csv', '아프리카_CO2_배출량_2010^F2022 (1) (1).csv', '아프리카_CO2_배출량_2010^F2022 (1).csv', '아프리카 재생에너지 비율 2011.csv', '아프리카_CO2_배출량_변화율_2011 (1).csv', 'ND-GAIN 적응 지수 2011.csv', '아프리카24개국_환경규제강도_2011.csv', '아프리카24개국_보호구역비율_2011 (1).csv', 'sample_data']


In [7]:
files = [
    '아프리카_CO2_배출량_변화율_2011.csv',
    '아프리카24개국_보호구역비율_2011.csv',
    '아프리카24개국_산림면적_2011.csv',
    '아프리카24개국_환경규제강도_2011.csv',
    'ND-GAIN 적응 지수 2011.csv',
    '아프리카 재생에너지 비율 2011.csv',
    '아프리카_CO2_배출량_2010^F2022 (1).csv'
]


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

dfs = []
for file in files:
    path = '/content/' + file
    df = pd.read_csv(path)
    # '2011' 컬럼 있는 파일은 해당 컬럼만 추출
    if '2011' in df.columns:
        sub = df[['국가', '2011']].copy()
        sub.columns = ['국가', file]
    else:
        # CO2배출량 연도별 파일은 '2011'만 뽑기
        if '국가' in df.columns and '2011' in df.columns:
            sub = df[['국가', '2011']].copy()
            sub.columns = ['국가', file]
        else:
            sub = df
    dfs.append(sub)

# 국가 기준 병합
from functools import reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on='국가'), dfs)

# 컬럼명 정리 (직접 변수명으로 교체)
df_merged.columns = [
    '국가',
    'CO2_증감률',
    '보호구역비율',
    '산림면적',
    '환경규제강도',
    'ND_GAIN_적응지수',
    '재생에너지비율',
    'CO2_배출량'
]

# 결측치 평균 대체
for col in df_merged.columns[1:]:
    df_merged[col] = df_merged[col].fillna(df_merged[col].mean())

X = df_merged.drop('국가', axis=1)

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA()
pca.fit(X_scaled)

# PC1 loading(가중치) 산출
pc1_loadings = pca.components_[0]
abs_loadings = np.abs(pc1_loadings)
weights = abs_loadings / abs_loadings.sum()

# 결과 출력
weights_df = pd.DataFrame({
    '지표명': X.columns,
    'PC1_loading': pc1_loadings,
    '가중치': weights
})

print(weights_df)


            지표명  PC1_loading       가중치
0       CO2_증감률    -0.308997  0.136221
1        보호구역비율    -0.335478  0.147895
2          산림면적    -0.447887  0.197450
3        환경규제강도     0.589804  0.260014
4  ND_GAIN_적응지수     0.000460  0.000203
5       재생에너지비율     0.103176  0.045485
6       CO2_배출량     0.482555  0.212733


In [9]:
# 표준화된 데이터 미리보기
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
print("표준화된 데이터(앞부분 5개 국가):")
print(X_scaled_df.head())


표준화된 데이터(앞부분 5개 국가):
    CO2_증감률    보호구역비율      산림면적    환경규제강도  ND_GAIN_적응지수   재생에너지비율   CO2_배출량
0  0.545230 -0.168232  0.404681  0.020662     -1.534799 -0.998877  1.569447
1  0.916285 -0.858417 -1.324727  1.067558      1.202248  0.931529  0.084659
2 -0.952409 -0.348001 -1.196902  0.553293      1.131771 -0.356948  0.759076
3 -0.509206  0.477812  0.029335  1.324690     -1.538265  0.954620 -0.174403
4  0.143948  1.085335  0.862147 -0.603802     -1.440059  1.379494  0.433643


In [10]:
print("PCA 기반 각 지표별 가중치:")
print(weights_df)


PCA 기반 각 지표별 가중치:
            지표명  PC1_loading       가중치
0       CO2_증감률    -0.308997  0.136221
1        보호구역비율    -0.335478  0.147895
2          산림면적    -0.447887  0.197450
3        환경규제강도     0.589804  0.260014
4  ND_GAIN_적응지수     0.000460  0.000203
5       재생에너지비율     0.103176  0.045485
6       CO2_배출량     0.482555  0.212733


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# (파일 병합 및 전처리 코드는 위와 동일)

# ... [생략] ...
# df_merged와 X, X_scaled, weights_df는 위 코드와 동일하게 만든다고 가정

# 표준화된 데이터 미리보기
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
print("표준화된 데이터(앞부분 5개 국가):")
print(X_scaled_df.head())

# PCA 가중치 미리보기
print("\nPCA 기반 각 지표별 가중치:")
print(weights_df)


표준화된 데이터(앞부분 5개 국가):
    CO2_증감률    보호구역비율      산림면적    환경규제강도  ND_GAIN_적응지수   재생에너지비율   CO2_배출량
0  0.545230 -0.168232  0.404681  0.020662     -1.534799 -0.998877  1.569447
1  0.916285 -0.858417 -1.324727  1.067558      1.202248  0.931529  0.084659
2 -0.952409 -0.348001 -1.196902  0.553293      1.131771 -0.356948  0.759076
3 -0.509206  0.477812  0.029335  1.324690     -1.538265  0.954620 -0.174403
4  0.143948  1.085335  0.862147 -0.603802     -1.440059  1.379494  0.433643

PCA 기반 각 지표별 가중치:
            지표명  PC1_loading       가중치
0       CO2_증감률    -0.308997  0.136221
1        보호구역비율    -0.335478  0.147895
2          산림면적    -0.447887  0.197450
3        환경규제강도     0.589804  0.260014
4  ND_GAIN_적응지수     0.000460  0.000203
5       재생에너지비율     0.103176  0.045485
6       CO2_배출량     0.482555  0.212733


In [12]:
from IPython.display import display

print("표준화된 데이터(앞부분 5개 국가):")
display(X_scaled_df.head())

print("\nPCA 기반 각 지표별 가중치:")
display(weights_df)


표준화된 데이터(앞부분 5개 국가):


,CO2_증감률,보호구역비율,산림면적,환경규제강도,ND_GAIN_적응지수,재생에너지비율,CO2_배출량
0,0.545230,-0.168232,0.404681,0.020662,-1.534799,-0.998877,1.569447
1,0.916285,-0.858417,-1.324727,1.067558,1.202248,0.931529,0.084659
2,-0.952409,-0.348001,-1.196902,0.553293,1.131771,-0.356948,0.759076
3,-0.509206,0.477812,0.029335,1.324690,-1.538265,0.954620,-0.174403
4,0.143948,1.085335,0.862147,-0.603802,-1.440059,1.379494,0.433643



PCA 기반 각 지표별 가중치:


,지표명,PC1_loading,가중치
0,CO2_증감률,-0.308997,0.136221
1,보호구역비율,-0.335478,0.147895
2,산림면적,-0.447887,0.197450
3,환경규제강도,0.589804,0.260014
4,ND_GAIN_적응지수,0.000460,0.000203
5,재생에너지비율,0.103176,0.045485
6,CO2_배출량,0.482555,0.212733


In [13]:
from sklearn.decomposition import PCA

pca = PCA()
X_pca = pca.fit_transform(X_scaled)


In [14]:
pc1_scores = X_pca[:, 0]


In [15]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
pc1_norm = minmax.fit_transform(pc1_scores.reshape(-1, 1))


In [16]:
result = pd.DataFrame({
    '국가': df_merged['국가'],
    '환경가치지수': pc1_norm.flatten()
})

print(result.head())


        국가    환경가치지수
0    Egypt  0.370801
1  Morocco  0.510768
2  Algeria  0.544332
3  Tunisia  0.428087
4  Nigeria  0.205524


In [17]:
# 1. 가중치(loading, 가중치) 결과 저장
weights_df.to_csv('/content/환경가치지수_가중치.csv', index=False, encoding='utf-8-sig')

# 2. 최종 환경가치지수 저장
result.to_csv('/content/환경가치지수_분석결과.csv', index=False, encoding='utf-8-sig')

# 3. Colab에서 다운로드
from google.colab import files
files.download('/content/환경가치지수_가중치.csv')
files.download('/content/환경가치지수_분석결과.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>